In [ ]:
### import packages
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
### read the data file
datafile_train=r'C:\Users\User\Documents\Data Science\Python\Hackathon\train.csv'
datafile_test=r'C:\Users\User\Documents\Data Science\Python\Hackathon\test.csv'
ld_train=pd.read_csv(datafile_train)
ld_test=pd.read_csv(datafile_test)

print(ld_train.shape)
print(ld_test.shape)

ld_train

list(zip(  ld_train.isnull().sum(), ld_train.nunique(), ld_train.dtypes, ld_train.columns))

1. covert origination_date and first_payment_date to date format
2. Loan pusrpose
3. categorical variable : source, financial_institution, 
    
 (0, 116058, dtype('int64'), 'loan_id') 
     remove
 (0, 3, dtype('O'), 'source')
     dummies
 (0, 19, dtype('O'), 'financial_institution') 
     dummies
 (0, 923, dtype('float64'), 'interest_rate'),
     features with ranges. 0to1, 1to2, 2to3, 3to4
 (0, 646, dtype('int64'), 'unpaid_principal_bal')
 (0, 140, dtype('int64'), 'loan_term'), = check the values
 (0, 3, dtype('O'), 'origination_date')
     create dummies, test and train date format alignment
 (0, 4, dtype('O'), 'first_payment_date'),
     create dummies, test and train date format alignment
 (0, 92, dtype('int64'), 'loan_to_value'),
 (0, 2, dtype('float64'), 'number_of_borrowers'),
         change value from 1, 2 to 1,0
         can create dummy but not created in iteration1
 (0, 58, dtype('float64'), 'debt_to_income_ratio'),
     create features with ranges : 1 to 5, 5 to 10
 (0, 221, dtype('float64'), 'borrower_credit_score'),
     create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
 (0, 3, dtype('O'), 'loan_purpose'),
     dummies
 (0, 14, dtype('float64'), 'insurance_percent'),
     create features with ranges 0to10. 11 to 20.. till 40
 (0, 216, dtype('float64'), 'co-borrower_credit_score'),
     create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
 (0, 2, dtype('float64'), 'insurance_type'),
 (0, 4, dtype('int64'), 'm1'),
 (0, 5, dtype('int64'), 'm2'),
 (0, 6, dtype('int64'), 'm3'),
 (0, 7, dtype('int64'), 'm4'),
 (0, 8, dtype('int64'), 'm5'),
 (0, 9, dtype('int64'), 'm6'),
 (0, 10, dtype('int64'), 'm7'),
 (0, 10, dtype('int64'), 'm8'),
 (0, 11, dtype('int64'), 'm9'),
 (0, 12, dtype('int64'), 'm10'),
 (0, 13, dtype('int64'), 'm11'),
 (0, 13, dtype('int64'), 'm12'),
 (0, 2, dtype('int64'), 'm13')

In [ ]:
#(0, 923, dtype('float64'), 'interest_rate') = features with ranges. 0to1, 1to2, 2to3, 3to4 to 8
ld_train['IR_0to1'] = np.where(ld_train['interest_rate']<=1,1,0)
ld_train['IR_1to2'] = np.where(ld_train['interest_rate'].between(1,2),1,0)
ld_train['IR_2to3'] = np.where(ld_train['interest_rate'].between(2,3),1,0)
ld_train['IR_3to4'] = np.where(ld_train['interest_rate'].between(3,4),1,0)
ld_train['IR_4to5'] = np.where(ld_train['interest_rate'].between(4,5),1,0)
ld_train['IR_5to6'] = np.where(ld_train['interest_rate'].between(5,6),1,0)
ld_train['IR_6to7'] = np.where(ld_train['interest_rate'].between(6,7),1,0)
ld_train.drop(['interest_rate'], axis=1)


#in test data
ld_test['IR_0to1'] = np.where(ld_test['interest_rate']<=1,1,0)
ld_test['IR_1to2'] = np.where(ld_test['interest_rate'].between(1,2),1,0)
ld_test['IR_2to3'] = np.where(ld_test['interest_rate'].between(2,3),1,0)
ld_test['IR_3to4'] = np.where(ld_test['interest_rate'].between(3,4),1,0)
ld_test['IR_4to5'] = np.where(ld_test['interest_rate'].between(4,5),1,0)
ld_test['IR_5to6'] = np.where(ld_test['interest_rate'].between(5,6),1,0)
ld_test['IR_6to7'] = np.where(ld_test['interest_rate'].between(6,7),1,0)

ld_test.drop(['interest_rate'], axis=1)

In [ ]:
#create dummy for top 7 values of loan_term
k=ld_train['loan_term'].value_counts()

for val in k.axes[0][0:7]:
    varname='loan_term_'+str(val)
    ld_train[varname]=np.where(ld_train['loan_term']==val,1,0)
    ld_test[varname]=np.where(ld_test['loan_term']==val,1,0)
del ld_train['loan_term']
del ld_test['loan_term']

In [ ]:
# (0, 58, dtype('float64'), 'debt_to_income_ratio') = create features with ranges : 1 to 5, 5 to 10 till 65
i=(0,5,10,15,20,25,30,35,40,45,50,55,60)

for var in i:
    var_name = 'DIR_'+str(var)+'to'+str(var+5)
    ld_train[var_name]=np.where(ld_train['debt_to_income_ratio'].between(var, var+5),1,0)
    ld_test[var_name]=np.where(ld_test['debt_to_income_ratio'].between(var, var+5),1,0)

del ld_train['debt_to_income_ratio']
del ld_test['debt_to_income_ratio']

In [ ]:
# (0, 221, dtype('float64'), 'borrower_credit_score') = create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
i=0

while i < 850:
    var_name = 'BCS_'+str(i)+'to'+str(i+50)
    ld_train[var_name]=np.where(ld_train['borrower_credit_score'].between(i, i+50),1,0)
    ld_test[var_name]=np.where(ld_test['borrower_credit_score'].between(i,i+50),1,0)
    i+=50

del ld_train['borrower_credit_score']
del ld_test['borrower_credit_score']

In [ ]:
# (0, 14, dtype('float64'), 'insurance_percent') = create features with ranges 0to10. 11 to 20.. till 40
i=0

while i < 40:
    i_from = i
    i_to = i+10
    var_name = 'IP_'+str(i_from)+'to'+str(i_to)
    ld_train[var_name]=np.where(ld_train['insurance_percent'].between(i_from, i_to),1,0)
    ld_test[var_name]=np.where(ld_test['insurance_percent'].between(i_from, i_to),1,0)
   # print(var_name)
    i+=10

del ld_train['insurance_percent']
del ld_test['insurance_percent']

In [ ]:
 #(0, 216, dtype('float64'), 'co-borrower_credit_score') = create features with ranges : 0 to 50, 51 to 100, 101 to 150 till 850
    
i=0

while i < 850:
    i_from = i
    i_to = i+50
    var_name = 'CBCS_'+str(i_from)+'to'+str(i_to)
    ld_train[var_name]=np.where(ld_train['co-borrower_credit_score'].between(i_from, i_to),1,0)
    ld_test[var_name]=np.where(ld_test['co-borrower_credit_score'].between(i_from, i_to),1,0)
    #print(var_name)
    i=i_to

del ld_train['co-borrower_credit_score']
del ld_test['co-borrower_credit_score']

Documentation reference for datetime format @https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
# (0, 3, dtype('O'), 'origination_date') =create dummies, test and train date format alignment
# (0, 4, dtype('O'), 'first_payment_date') = create dummies, test and train date format alignment
#convert the columns into date time
for col in ['origination_date','first_payment_date']:
    ld_train[col]=pd.to_datetime(ld_train[col],infer_datetime_format=True)
    
for col in ['origination_date']:
    ld_test[col]=pd.to_datetime(ld_test[col],infer_datetime_format=True)

for col in ['first_payment_date']:
    ld_test['first_payment_date']=pd.to_datetime(ld_test[col],infer_datetime_format=True, format='%b-%y')

In [ ]:
#create feature having difference between origination date and first payment date
#droping both originating and first payment date column
ld_train['diff_date_origin_firstPay']=pd.to_numeric(ld_train['first_payment_date']-ld_train['origination_date'])/(60*60*24*1000000000)
ld_test['diff_date_origin_firstPay']=pd.to_numeric(ld_test['first_payment_date']-ld_test['origination_date'])/(60*60*24*1000000000)

del ld_train['origination_date']
del ld_test['origination_date']
del ld_train['first_payment_date']
del ld_test['first_payment_date']

In [ ]:
#create dummy columns
for col in ['source', 'financial_institution','loan_purpose']:
    
    temp=pd.get_dummies(ld_train[col],prefix=col,drop_first=True)
    ld_train=pd.concat([temp,ld_train],1)
    ld_train.drop([col],1,inplace=True)
    
    temp=pd.get_dummies(ld_test[col],prefix=col,drop_first=True)
    ld_test=pd.concat([temp,ld_test],1)
    ld_test.drop([col],1,inplace=True)

print(ld_test.shape)
print(ld_train.shape)

### train and test dataset creation

In [ ]:
#from sklearn.cross_validation import train_test_split,KFold
x=ld_train.drop(['loan_id','m13'],1)
y=ld_train['m13']


### Model creation

#### Random Forest

In [ ]:
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost.sklearn import XGBClassifier

In [ ]:
gbm_params = { "n_estimators":[200,300,400,500,600,700],
              "learning_rate":[0.01, 0.03, 0.05, 0.07, 0.1,0.4,0.8,1],
              "max_depth":[1,2,3,4,5,6],
              "subsample":[0.5,0.8,1],
              "max_features":[5,10,15,20,30,45]
}



In [ ]:
gbm = GradientBoostingClassifier()

In [ ]:
random_search = RandomizedSearchCV(gbm, param_distributions=gbm_params, n_iter=150,scoring='f1',cv=10, verbose=1, n_jobs=-1)

In [ ]:
random_search.fit(x, y)

Fitting 10 folds for each of 500 candidates, totalling 5000 fits


In [31]:
r_forest=random_search.best_estimator_
r_forest

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03, loss='deviance', max_depth=3,
              max_features=10, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=0.8, verbose=0,
              warm_start=False)

In [21]:
##Report function
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [32]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.504 (std: 0.03621)
Parameters: {'subsample': 0.8, 'n_estimators': 500, 'max_features': 10, 'max_depth': 3, 'learning_rate': 0.03}

Model with rank: 2
Mean validation score: 0.494 (std: 0.03168)
Parameters: {'subsample': 1, 'n_estimators': 400, 'max_features': 30, 'max_depth': 5, 'learning_rate': 0.01}

Model with rank: 3
Mean validation score: 0.494 (std: 0.04310)
Parameters: {'subsample': 0.8, 'n_estimators': 500, 'max_features': 45, 'max_depth': 1, 'learning_rate': 0.01}

Model with rank: 4
Mean validation score: 0.491 (std: 0.05546)
Parameters: {'subsample': 0.5, 'n_estimators': 500, 'max_features': 30, 'max_depth': 1, 'learning_rate': 0.01}

Model with rank: 5
Mean validation score: 0.427 (std: 0.05514)
Parameters: {'subsample': 1, 'n_estimators': 500, 'max_features': 5, 'max_depth': 4, 'learning_rate': 0.1}



In [33]:
r_forest.fit(x,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03, loss='deviance', max_depth=3,
              max_features=10, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=0.8, verbose=0,
              warm_start=False)

In [34]:
prediction=np.where(r_forest.predict(ld_test.drop(['loan_id'],1))==1,1,0)
submission=pd.DataFrame(list(zip(ld_test['loan_id'],list(prediction))),
                       columns=['loan_id','m13'])

In [35]:
submission.head(4)
print(ld_test.shape)

(35866, 106)


In [36]:
submission.to_csv('sample_submission_2008_Boost2.csv',index=False)